<a href="https://colab.research.google.com/github/sanimesa/genai/blob/main/notebooks/BigQuery_Vector_Search_vectorize_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstrate loading text embeddings into a BigQuery dataset
This example uses a sentence-transformer model and a Huggingface Token needs to be added to a secret called. HF_TOKEN.
https://huggingface.co/docs/hub/security-tokens

If you are planning to embed large amounts of data,
Select Runtime -> Change runtime type to use T4 GPU.


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.8 MB/s eta 0:00:00


#### Authorize the user

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.colab import userdata
import os

#### Define the constants as desired

In [ ]:
project_id = userdata.get('PROJECT_ID')
dataset_id = 'genai'
table_name = 'bigquery-public-data.hacker_news.full'
destination_table = f'{dataset_id}.hacker_news_embedded3'
search_table = f'{dataset_id}.hacker_news_search3'

TOTAL_ROWS = 20000
PAGE_SIZE = 2000

#### Function to create embeddings from a list of texts

In [ ]:
from typing import List
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embeddings(texts: List[str])  -> List[List[float]]:
  return model.encode(texts).tolist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
%%time
embeddings = get_embeddings(['this is an example text']*2000)
print(type(embeddings))
print(len(embeddings))
print(len(embeddings[0]))

<class 'list'>
2000
384
CPU times: user 5.54 s, sys: 38.2 ms, total: 5.58 s
Wall time: 5.59 s


#### Process a dataframe: must contain a column called 'text' which will  be embedded and added to the dataframe. Finally, the dataframe will be persisted into BigQuery

In [ ]:
def process_dataframe(df):
    try:
      texts = df['text'].tolist()
      embeddings = get_embeddings(texts)
      df['text_embedding'] = get_embeddings(texts)
      df.to_gbq(destination_table, project_id=project_id, if_exists='append')
    except Exception as e:
        print(embeddings)
        print('an exception occured: ', e)


#### Read a BigQuery table and process in batches of PAGE_SIZE

In [ ]:
from google.cloud import bigquery

def read_bigquery_table(table_name: str, year: int = 2022):
    client = bigquery.Client(project=project_id)
    query = f"SELECT id, timestamp, text FROM `{table_name}` WHERE extract(year FROM timestamp) = {year} and text is not null LIMIT {TOTAL_ROWS}"
    job = client.query(query)
    result = job.result(page_size=PAGE_SIZE)

    for df in result.to_dataframe_iterable():
        process_dataframe(df)

In [ ]:
%%time
read_bigquery_table(table_name, 2015)

100%|██████████| 1/1 [00:00<00:00, 9238.56it/s]


#### Create a search table: this will contain the embedding to be searched for

In [ ]:
%%time
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

search_query = 'social media'
embeddings = get_embeddings([search_query])

query = """
  CREATE OR REPLACE TABLE {search_table} AS
  SELECT '{search_query}' search_query,  {embeddings} AS text_embedding;
""".format(search_table=search_table, search_query=search_query, embeddings=str(embeddings[0]))

query_job = client.query(query)
query_job.result()

CPU times: user 91.2 ms, sys: 7.99 ms, total: 99.2 ms
Wall time: 2.59 s
